In [152]:
import requests
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd 
import argparse

In [159]:
def embajadas():
    response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
    results = response.json()
    embajadas = pd.json_normalize(results['@graph'])
    embajadas["Type of place"] = "Embajadas y consulados"
    embajadas["Dirección"] = embajadas["address.street-address"]
    embajadas["location.latitude"] = pd.to_numeric(embajadas["location.latitude"],errors = 'coerce')
    embajadas["location.longitude"] = pd.to_numeric(embajadas["location.longitude"],errors = 'coerce')
    embajadas_clean = embajadas[["title","Type of place", "Dirección","location.longitude","location.latitude"]]
    embajadas_clean = embajadas_clean.rename(columns={"title": "Place of interest", "Dirección": "Place address", "location.longitude": "Longitud_inicial", "location.latitude": "Latitud_inicial"}, errors="raise")
    return embajadas_clean

def estaciones():
    estaciones = pd.read_json("../data/bicimad2.json", orient='records')
    geometry_coordinates = estaciones["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    estaciones = pd.concat([estaciones, geometry_coordinates], axis=1)
    estaciones["LONGITUD"] = pd.to_numeric(estaciones["LONGITUD"],errors = 'coerce')
    estaciones["LATITUD"] = pd.to_numeric(estaciones["LATITUD"],errors = 'coerce')
    estaciones_clean = estaciones[["name","address","LONGITUD","LATITUD"]]
    estaciones_clean = estaciones_clean.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Longitud_final", "LATITUD": "Latitud_final"})
    return estaciones_clean


In [160]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)
    


In [175]:
#embajadas = pd.read_json('../data/embajadas.json')
#estaciones = pd.read_json('../data/estaciones.json')

In [176]:
def mercator_emb():
    embajadas_clean["mercator_start"] = embajadas_clean.apply(lambda x: to_mercator(x['Longitud_inicial'], x['Latitud_inicial']), axis=1)
    return embajadas_clean

def mercator_est():
    estaciones_clean["mercator_finish"] = estaciones_clean.apply(lambda x: to_mercator(x["Latitud_final"], x["Longitud_final"]), axis=1)
    return estaciones_clean

In [189]:
def merge():
    df_resultado = pd.merge(embajadas_clean,estaciones_clean, how='cross')
    return df_resultado

In [190]:
def apply_distancia():
    df_resultado["Distancia"] = df_resultado.apply(lambda x: distance_meters(x["mercator_start"], x["mercator_finish"]), axis=1)
    return df_resultado

In [191]:
def minimum():
    df_minimo = df_resultado[df_resultado["Place of interest"] == input('Pon el lugar de interés: ')]
    df_minimum = df_minimo[df_minimo['Distancia'] == df_minimo['Distancia'].min()]
    minimum_mod = df_minimum[["Place of interest","Type of place", "Place address","BiciMAD station","Station location"]]
    return minimum_mod

In [192]:
minimum()

Pon el lugar de interés: Consulado de Argentina


,Place of interest,Type of place,Place address,BiciMAD station,Station location
223,Consulado de Argentina,Embajadas y consulados,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,Pavones,Calle Hacienda de Pavones nº 342


In [193]:
def all_minimum():
    return df_resultado.sort_values(by = "Distancia", ascending = True).groupby('Place of interest')['Place address', 'Distancia','BiciMAD station', 'Station location'].nth(0).drop(["Distancia"], axis = "columns")

In [194]:
all_minimum()

/Users/Andrea/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Place address,BiciMAD station,Station location
Place of interest,,,
Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,Pavones,Calle Hacienda de Pavones nº 342
Consulado de Belice,CALLE TALAVERA 9,Pavones,Calle Hacienda de Pavones nº 342
Consulado de Bolivia,CALLE AVIADOR LINDBERGH 3,Pavones,Calle Hacienda de Pavones nº 342
Consulado de Brasil,CALLE GOYA 5 y 7 pasaje,Pavones,Calle Hacienda de Pavones nº 342
Consulado de Bélgica,PASEO CASTELLANA 18 PLANTA 6 Izquierda,Pavones,Calle Hacienda de Pavones nº 342
...,...,...,...
Embajada de Vietnam,AVENIDA ALFONSO XIII 54,Pavones,Calle Hacienda de Pavones nº 342
Embajada de Yemen,PASEO CASTELLANA 117 PLANTA 8 D,Pavones,Calle Hacienda de Pavones nº 342
Embajada de la República Popular Democrática de Corea,CALLE DARIO APARICIO 43,Pavones,Calle Hacienda de Pavones nº 342


In [195]:
parser = argparse.ArgumentParser()
parser.add_argument('-a', type= str, help= 'Lugar de interés')

                    
                    
                    

_StoreAction(option_strings=['-a'], dest='a', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Lugar de interés', metavar=None)

In [ ]:
args = parser.parse_args()
print(args)

In [ ]:
# Main pipeline function
def main():
    print('--//--- starting application ---//--')
    print('\n')
    parser = argparse.ArgumentParser()
    #parser.add_argument('--p', type=str,help= 'Enter a place of interest')
    parser.add_argument('--function', type=str,help= "'minimum' or 'all_places'")
    args=parser.parse_args()
    sys.stdout.write(str(minimum(args)))
# Pipeline execution
if __name__ == '__main__':
    main(function(args))